In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Coding the self-attention architecture

In [4]:
class SelfAttn(nn.Module):
  def __init__(self, dim_model = 2,
               row_dim = 0,
               col_dim = 1):
    ## dim_model = the number of embedding values per token.
    ##           Because we want to be able to do the math by hand, we've
    ##           the default value for d_model=2.
    ##           However, in "Attention Is All You Need" d_model=512
    ##
    ## row_dim, col_dim = the indices we should use to access rows or columns

    super().__init__()

    ## Initialize the Weights (W) that we'll use to create the
    ## query (q), key (k) and value (v) for each token
    ## NOTE: A lot of implementations include bias terms when
    ##       creating the the queries, keys, and values, but
    ##       the original manuscript that described Attention,
    ##       "Attention Is All You Need" did not, so we won't either


    self.W_q = nn.Linear(in_features = dim_model,
                        out_features = dim_model,
                        bias = False)
    self.W_k = nn.Linear(in_features = dim_model,
                        out_features = dim_model,
                        bias = False)
    self.W_v = nn.Linear(in_features = dim_model,
                        out_features = dim_model,
                        bias = False)

    self.row_dim = row_dim
    self.col_dim = col_dim

  def forward(self, token_embeddings):

      ## Create the query, key and values using the encoding numbers
      ## associated with each token (token encodings)


    q = self.W_q(token_embeddings) #tok_emb x Wq = Q
    k = self.W_k(token_embeddings) #tok_emb x Wk = K
    v = self.W_v(token_embeddings) #tok_emb x Wv = V

    sim_score = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim)) ## Compute similarities scores: (q * k^T)

    scaled_sim_score = sim_score / (k.shape[self.col_dim] ** 0.5) ## Scale the similarities by dividing by sqrt(k.col_dim)

    ## Apply softmax to determine what percent of each tokens' value to
    ## use in the final attention values.

    attn_percent = F.softmax(scaled_sim_score, dim=self.col_dim)

    ## Scale the values by their associated percentages and add them up.
    attn_scores = torch.matmul(attn_percent, v)

    return attn_scores






Testing out the caluclation of self-attention

In [5]:
embeddings_matrix = torch.tensor([[1.16, 0.23],
                                  [0.57, 1.36],
                                  [4.41, -2.16]])

In [6]:
torch.manual_seed(42) #ensure reproducibility

In [7]:
sa = SelfAttn(dim_model = 2,
              row_dim = 0,
              col_dim = 1)

In [8]:
sa(embeddings_matrix)

tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)

validating the math

In [9]:
sa.W_q.weight.transpose(0,1) #Wq

tensor([[ 0.5406, -0.1657],
        [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>)

In [10]:
sa.W_k.weight.transpose(0,1) #Wk

tensor([[-0.1549, -0.3443],
        [ 0.1427,  0.4153]], grad_fn=<TransposeBackward0>)

In [11]:
sa.W_v.weight.transpose(0,1) #Wv

tensor([[ 0.6233,  0.6146],
        [-0.5188,  0.1323]], grad_fn=<TransposeBackward0>)

In [12]:
sa.W_q(embeddings_matrix) #q

tensor([[ 0.7621, -0.0428],
        [ 1.1063,  0.7890],
        [ 1.1164, -2.1336]], grad_fn=<MmBackward0>)

In [13]:
sa.W_k(embeddings_matrix) #k

tensor([[-0.1469, -0.3038],
        [ 0.1057,  0.3685],
        [-0.9914, -2.4152]], grad_fn=<MmBackward0>)

In [14]:
sa.W_v(embeddings_matrix) #v

tensor([[ 0.6038,  0.7434],
        [-0.3502,  0.5303],
        [ 3.8695,  2.4246]], grad_fn=<MmBackward0>)